<a href="https://colab.research.google.com/github/jjome/myhome/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import division
import os
import time
from glob import glob
import tensorflow as tf
import numpy as np
from collections import namedtuple
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, LeakyReLU, Activation
from tensorflow.keras.models import Sequential  # Sequential 클래스 임포트 추가
import numpy as np
import os
from glob import glob
import time
from collections import namedtuple

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !git clone https://github.com/hyunbo9/face_aging_with_CycleGan.git
!git clone https://github.com/jjome/myhome.git

In [ ]:
%cd myhome

In [ ]:
!pip install imageio Pillow

In [ ]:
from module import *

In [69]:
class cyclegan:
    def __init__(self, checkpoint_dir, test_dir, dataset_dir, which_direction):
        # 초기화 메서드. 클래스의 인스턴스를 생성할 때 호출됩니다.
        self.batch_size = 1  # 배치 크기
        self.image_size = 256  # 이미지 크기 (256x256)
        self.input_c_dim = 3  # 입력 채널 수 (RGB 이미지이므로 3)
        self.output_c_dim = 3  # 출력 채널 수 (RGB 이미지이므로 3)
        self.L1_lambda = 10.0  # L1 손실의 가중치
        self.fine_size = 256  # 최종 이미지 크기
        self.ngf = 64  # 생성자의 첫 번째 레이어의 필터 수
        self.ndf = 64  # 판별자의 첫 번째 레이어의 필터 수
        self.output_nc = 3  # 출력 채널 수
        self.max_size = 50  # 이미지 풀의 최대 크기
        self.beta1 = 0.5  # Adam 옵티마이저의 베타1 값
        self.epoch = 200  # 총 학습 에폭 수
        self.epoch_step = 100  # 학습률 감소 단계
        self.train_size = int(1e8)  # 훈련 데이터 크기
        self.lr_init = 0.0002  # 초기 학습률
        self.load_size = 286  # 로드할 이미지 크기
        self.save_freq = 500  # 체크포인트 저장 빈도
        self.continue_train = True  # 학습 재개 여부

        self.checkpoint_dir = checkpoint_dir  # 체크포인트 디렉토리
        self.dataset_dir = dataset_dir  # 데이터셋 디렉토리
        self.test_dir = test_dir  # 테스트 디렉토리

        self.discriminator = self.build_discriminator()  # 판별자 생성
        self.generator = self.build_generator()  # 생성자 생성

        self.original_GAN_loss = self.mae_criterion  # GAN 손실 기준 설정

        self.which_direction = which_direction  # 변환 방향 (AtoB 또는 BtoA)

        # 옵션(namedtuple)을 정의하고 설정
        OPTIONS = namedtuple('OPTIONS', 'batch_size image_size gf_dim df_dim output_c_dim')
        self.options = OPTIONS._make((self.batch_size, self.fine_size, self.ngf, self.ndf, self.output_nc,))

        self._build_model()  # 모델 구성
        self.checkpoint = tf.train.Checkpoint(generator=self.generator, discriminator=self.discriminator)  # 체크포인트 설정
        self.checkpoint_manager = tf.train.CheckpointManager(self.checkpoint, self.checkpoint_dir, max_to_keep=5)  # 체크포인트 매니저 설정

    def mae_criterion(self, x, y):
        # 절대 오차 평균(MAE) 손실 함수
        return tf.reduce_mean(tf.abs(x - y))

    def abs_criterion(self, x, y):
        # 절대 오차 평균(MAE) 손실 함수 (중복)
        return tf.reduce_mean(tf.abs(x - y))

    def _build_model(self):
        # CycleGAN 모델 구성
        self.real_A = tf.keras.Input(shape=(self.image_size, self.image_size, self.input_c_dim), name='real_A')
        self.real_B = tf.keras.Input(shape=(self.image_size, self.image_size, self.input_c_dim), name='real_B')

        self.fake_B = self.generator(self.real_A)
        self.fake_A_ = self.generator(self.fake_B)
        self.fake_A = self.generator(self.real_B)
        self.fake_B_ = self.generator(self.fake_A)

        self.DB_fake = self.discriminator(self.fake_B)
        self.DA_fake = self.discriminator(self.fake_A)

        # 생성자 손실 계산
        self.g_loss = self.original_GAN_loss(self.DA_fake, tf.ones_like(self.DA_fake)) \
            + self.original_GAN_loss(self.DB_fake, tf.ones_like(self.DB_fake)) \
            + self.L1_lambda * self.abs_criterion(self.real_A, self.fake_A_) \
            + self.L1_lambda * self.abs_criterion(self.real_B, self.fake_B_)

        self.fake_A_sample = tf.keras.Input(shape=(self.image_size, self.image_size, self.input_c_dim), name='fake_A_sample')
        self.fake_B_sample = tf.keras.Input(shape=(self.image_size, self.image_size, self.input_c_dim), name='fake_B_sample')

        self.DB_real = self.discriminator(self.real_B)
        self.DA_real = self.discriminator(self.real_A)
        self.DB_fake_sample = self.discriminator(self.fake_B_sample)
        self.DA_fake_sample = self.discriminator(self.fake_A_sample)

        # 판별자 손실 계산
        self.db_loss_real = self.original_GAN_loss(self.DB_real, tf.ones_like(self.DB_real))
        self.db_loss_fake = self.original_GAN_loss(self.DB_fake_sample, tf.zeros_like(self.DB_fake_sample))
        self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2

        self.da_loss_real = self.original_GAN_loss(self.DA_real, tf.ones_like(self.DA_real))
        self.da_loss_fake = self.original_GAN_loss(self.DA_fake_sample, tf.zeros_like(self.DA_fake_sample))
        self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2

        self.d_loss = self.da_loss + self.db_loss

        self.test_A = tf.keras.Input(shape=(self.image_size, self.image_size, self.input_c_dim), name='test_A')
        self.test_B = tf.keras.Input(shape=(self.image_size, self.image_size, self.input_c_dim), name='test_B')
        self.testB = self.generator(self.test_A)
        self.testA = self.generator(self.test_B)

        self.d_vars = self.discriminator.trainable_variables
        self.g_vars = self.generator.trainable_variables


    def build_generator(self):
        # 생성자 모델 구성 (네트워크 깊이 조정 예시)
        model = Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(self.image_size, self.image_size, self.input_c_dim)))
        model.add(Conv2D(64, (7, 7), padding='same'))
        model.add(Activation('relu'))
        # 추가 레이어를 여기서 추가
        for _ in range(4):  # 네트워크 깊이 증가
            model.add(Conv2D(64, (3, 3), padding='same'))
            model.add(Activation('relu'))
        model.add(Conv2D(self.output_nc, (7, 7), padding='same'))
        model.add(Activation('tanh'))
        return model

    def build_discriminator(self):
        # 판별자 모델 구성 (네트워크 깊이 조정 예시)
        model = Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(self.image_size, self.image_size, self.input_c_dim)))
        model.add(Conv2D(64, (4, 4), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # 추가 레이어를 여기서 추가
        for _ in range(3):  # 네트워크 깊이 증가
            model.add(Conv2D(64, (4, 4), padding='same'))
            model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(1, (4, 4), padding='same'))
        return model



    @tf.function
    def train_step(self, real_A, real_B, optimizer):
        with tf.GradientTape(persistent=True) as tape:
            fake_B = self.generator(real_A, training=True)
            fake_A = self.generator(real_B, training=True)

            DB_fake = self.discriminator(fake_B, training=True)
            DA_fake = self.discriminator(fake_A, training=True)

            g_loss = self.original_GAN_loss(DA_fake, tf.ones_like(DA_fake)) \
                + self.original_GAN_loss(DB_fake, tf.ones_like(DB_fake)) \
                + self.L1_lambda * self.abs_criterion(real_A, self.generator(fake_B)) \
                + self.L1_lambda * self.abs_criterion(real_B, self.generator(fake_A))

            DB_real = self.discriminator(real_B, training=True)
            DA_real = self.discriminator(real_A, training=True)

            db_loss_real = self.original_GAN_loss(DB_real, tf.ones_like(DB_real))
            db_loss_fake = self.original_GAN_loss(DB_fake, tf.zeros_like(DB_fake))
            db_loss = (db_loss_real + db_loss_fake) / 2

            da_loss_real = self.original_GAN_loss(DA_real, tf.ones_like(DA_real))
            da_loss_fake = self.original_GAN_loss(DA_fake, tf.zeros_like(DA_fake))
            da_loss = (da_loss_real + da_loss_fake) / 2

            d_loss = da_loss + db_loss

        d_gradients = tape.gradient(d_loss, self.d_vars)
        g_gradients = tape.gradient(g_loss, self.g_vars)
        optimizer.apply_gradients(zip(d_gradients, self.d_vars))
        optimizer.apply_gradients(zip(g_gradients, self.g_vars))

        return d_loss, g_loss



    def train(self):
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr_init, beta_1=self.beta1)
        optimizer.build(self.d_vars + self.g_vars)  # 전체 모델 변수 목록 제공

        for epoch in range(self.epoch):
            print(f"Epoch [{epoch+1}/{self.epoch}] 시작")
            dataA = glob('{}/*.*'.format(self.dataset_dir + '/trainA'))
            dataB = glob('{}/*.*'.format(self.dataset_dir + '/trainB'))
            np.random.shuffle(dataA)
            np.random.shuffle(dataB)
            batch_idxs = min(len(dataA), len(dataB)) // self.batch_size
            for idx in range(0, batch_idxs):
                batch_files = list(zip(dataA[idx * self.batch_size:(idx + 1) * self.batch_size],
                                      dataB[idx * self.batch_size:(idx + 1) * self.batch_size]))
                batch_images = [self.load_train_data(batch_file, self.load_size, self.fine_size) for batch_file in batch_files]
                batch_images = np.array(batch_images).astype(np.float32)
                real_A, real_B = batch_images[:, :, :, :self.input_c_dim], batch_images[:, :, :, self.input_c_dim:]

                d_loss, g_loss = self.train_step(real_A, real_B, optimizer)

                if idx % 10 == 0:
                    print(f"Batch [{idx+1}/{batch_idxs}] - d_loss: {d_loss.numpy()}, g_loss: {g_loss.numpy()}")

                if (epoch * batch_idxs + idx) % self.save_freq == 0:
                    self.checkpoint_manager.save()
                    print(f"체크포인트 저장됨 (Epoch: {epoch+1}, Batch: {idx+1})")

            print(f"Epoch [{epoch+1}/{self.epoch}] 종료")



    def load_train_data(self, batch_file, load_size, fine_size):
        # 학습 데이터를 로드하고 전처리
        img_A = tf.image.decode_jpeg(tf.io.read_file(batch_file[0]))
        img_B = tf.image.decode_jpeg(tf.io.read_file(batch_file[1]))
        img_A = tf.image.resize(img_A, [load_size, load_size])
        img_B = tf.image.resize(img_B, [load_size, load_size])
        img_A = (img_A / 127.5) - 1
        img_B = (img_B / 127.5) - 1
        img_A = tf.image.random_crop(img_A, [fine_size, fine_size, self.input_c_dim])
        img_B = tf.image.random_crop(img_B, [fine_size, fine_size, self.input_c_dim])
        return np.concatenate((img_A, img_B), axis=2)


    def test(self):
        if self.which_direction == 'AtoB':
            sample_files = glob('{}/*.*'.format(self.dataset_dir + '/testA'))
        elif self.which_direction == 'BtoA':
            sample_files = glob('{}/*.*'.format(self.dataset_dir + '/testB'))
        else:
            raise Exception('AtoB or BtoA must be specified.')

        latest = self.checkpoint_manager.latest_checkpoint
        if (latest):
            self.checkpoint.restore(latest).expect_partial()
            print("체크포인트가 복원됨:", latest)
        else:
            print("체크포인트가 없습니다.")

        for sample_file in sample_files:
            print('이미지 처리 중: ' + sample_file)
            sample_image = [self.load_test_data(sample_file, self.fine_size)]
            sample_image = np.array(sample_image).astype(np.float32)
            sample_image = tf.convert_to_tensor(sample_image)  # 추가된 부분: 텐서로 변환

            if self.which_direction == 'AtoB':
                fake_img = self.generator(sample_image, training=False)
            else:
                fake_img = self.generator(sample_image, training=False)

            image_path = os.path.join(self.test_dir, '{0}_{1}'.format(self.which_direction, os.path.basename(sample_file)))
            self.save_images(fake_img, [1, 1], image_path)
            print('이미지 저장됨: ' + image_path)


    def load_test_data(self, image_path, fine_size):
        # 테스트 데이터를 로드하고 전처리
        img = tf.image.decode_jpeg(tf.io.read_file(image_path))
        img = tf.image.resize(img, [fine_size, fine_size])
        img = (img / 127.5) - 1
        return img


    def save_images(self, images, size, image_path):
        # 이미지를 저장
        images = (images + 1) * 127.5
        images = np.clip(images, 0, 255).astype(np.uint8)
        images = tf.cast(images, tf.uint8)
        images = images.numpy()

        # PIL을 사용하여 명암 반전
        for i, img in enumerate(images):
            img_pil = Image.fromarray(img)
            img_inverted = ImageOps.invert(img_pil)
            img_inverted.save(f"{image_path}_{i}.png")



In [70]:
# Assuming cyclegan is defined elsewhere or imported
# from cyclegan import cyclegan

def main(which_direction, phase):
    checkpoint_dir = './checkpoint/face_256'             # 체크포인트 경로
    test_dir = './test'                          # 테스트 이미지가 저장되는 경로
    dataset_dir = './datasets/face'                         # 데이터셋 위치    trainA trainB testA testB
    # phase = "test"  # or train
    # which_direction = "BtoA"        # or AtoB .  테스트시 변환 방향
    # 이미지 저장은 test 경로를 따로 만들어서 함.

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    # GPU 설정
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    # 모델 초기화
    model = cyclegan(checkpoint_dir=checkpoint_dir, test_dir=test_dir,
                     dataset_dir=dataset_dir, which_direction=which_direction)

    if phase == 'train':
        print("훈련 시작")
        model.train()
    elif phase == "test":
        print("테스트 시작")
        model.test()
    else:
        print("train??? test???? 둘중하나는 고르세요.")

In [1]:
if __name__ == '__main__':
    phase = "train"
    which_direction = "BtoA"
    main(which_direction, phase)

NameError: name 'main' is not defined

In [ ]:
if __name__ == '__main__':
    phase = "train"
    which_direction = "AtoB"
    main(which_direction, phase)

In [ ]:
if __name__ == '__main__':
    phase = "test"
    which_direction = "BtoA"
    main(which_direction, phase)

In [ ]:
if __name__ == '__main__':
    phase = "test"
    which_direction = "AtoB"
    main(which_direction, phase)